# Notas

In [ ]:
# Para los clientes de Locales Frontera que fueron impactados por los Ofertatas 4 y 7 buscamos
# Entender su comportamiento (frecuencia, RFM, consumo) previo al impacto
# Si se comportaron de acuerdo a lo esperado por la promo

# Importamos librerias

In [2]:
import pandas as pd
import numpy as np
import snowflake.connector
import json
from snowflake.connector.pandas_tools import write_pandas
from datetime import datetime
from datetime import timedelta
from dateutil.relativedelta import relativedelta
import os
pd.options.mode.chained_assignment = None

In [3]:
while True:
    try:

        f = open('credentials.json')
        data_pass = json.load(f)

        pass_ = input("INGRESAR PASSCODE:")

        ctx = snowflake.connector.connect(
            user = data_pass['snow']['USER'],
            password = data_pass['snow']['PASS'],
            account = data_pass['snow']['ACCOUNT'],
            passcode = pass_,
            database = 'SANDBOX_PLUS',
            schema = 'DWH'
        )

        cursor = ctx.cursor()

        print('Connected')

        break

    except:
        print('Incorrect Password - provide again')

Connected


# Clientes Frontera

In [15]:
query = '''
WITH LOCAL_DOMINANTE AS
    (
    SELECT
        DISTINCT
        SOCI_SOCI_ID,
        DOMINANTE,
        GEOG_LOCL_DESC
    FROM
        (
        SELECT
            FFM.SOCI_SOCI_ID,
            LGL.GEOG_LOCL_COD AS DOMINANTE,
            LGL.GEOG_LOCL_DESC,
            SUM(FV.VNTA_IMPORTE_SIN_IVA) AS VENTAS,
            ROW_NUMBER() OVER (PARTITION BY SOCI_SOCI_ID ORDER BY VENTAS DESC) AS RNK
        FROM
            MSTRDB.DWH.FT_VENTAS AS FV
            INNER JOIN MSTRDB.DWH.FT_FDLN_MOVIMIENTOS AS FFM ON FV.TICKET = FFM.TICKET
            INNER JOIN MSTRDB.DWH.LU_GEOG_LOCAL AS LGL ON FV.GEOG_LOCL_ID = LGL.GEOG_LOCL_ID
            INNER JOIN MSTRDB.DWH.LU_GEOG_DEPTO AS LGD ON LGD.GEOG_DPTO_ID = LGL.GEOG_DPTO_ID
            INNER JOIN MSTRDB.DWH.LU_ARTC_ARTICULO AS LAA ON LAA.ARTC_ARTC_ID = FV.ARTC_ARTC_ID
        WHERE
            FV.TIEM_DIA_ID BETWEEN  DATEADD(MONTH, -6, CURRENT_DATE()) AND CURRENT_DATE()
            AND LGL.GEOG_UNNG_ID = 2
            AND LAA.ARTC_SECC_ID <> 42
            AND FFM.FDLN_MOVT_TIPO LIKE '%RP%'
        GROUP BY
            ALL
        )
    WHERE
        RNK = 1
        AND DOMINANTE IN (111, 152, 156, 158, 159, 160, 161, 162, 316, 125, 127, 317)
    )

SELECT
    *
FROM
    SANDBOX_PLUS.DWH.SEGMENTOS_OFERTATA
WHERE
    SOCI_SOCI_ID IN (SELECT SOCI_SOCI_ID FROM LOCAL_DOMINANTE)
'''

cursor.execute(query)
clientes = cursor.fetch_pandas_all()

clientes['SOCI_SOCI_ID'] = clientes['SOCI_SOCI_ID'].astype(str)
print(clientes.shape[0])
clientes.head(2)

343762


,OFERTATA,FECHA,SUBCLASE,PERFIL_CLIENTE,SOCI_SOCI_ID,CLIE_TIPO_DOC,CLIE_CLIE_NDOC,CORREO,CELULAR
0,Ofertata 4,2024-01-25,1500030002,ACTIVO,100943,C,39201653,ileanalallanes79@hotmail.com,96122860
1,Ofertata 4,2024-01-25,1600090003,ACTIVO,99234,C,48627175,micalamarca07@gmail.com,99862459


# 1. Comportamiento

In [99]:
query = '''
WITH LOCAL_DOMINANTE AS
    (
    SELECT
        DISTINCT
        SOCI_SOCI_ID,
        DOMINANTE,
        GEOG_LOCL_DESC
    FROM
        (
        SELECT
            FFM.SOCI_SOCI_ID,
            LGL.GEOG_LOCL_COD AS DOMINANTE,
            LGL.GEOG_LOCL_DESC,
            SUM(FV.VNTA_IMPORTE_SIN_IVA) AS VENTAS,
            ROW_NUMBER() OVER (PARTITION BY SOCI_SOCI_ID ORDER BY VENTAS DESC) AS RNK
        FROM
            MSTRDB.DWH.FT_VENTAS AS FV
            INNER JOIN MSTRDB.DWH.FT_FDLN_MOVIMIENTOS AS FFM ON FV.TICKET = FFM.TICKET
            INNER JOIN MSTRDB.DWH.LU_GEOG_LOCAL AS LGL ON FV.GEOG_LOCL_ID = LGL.GEOG_LOCL_ID
            INNER JOIN MSTRDB.DWH.LU_GEOG_DEPTO AS LGD ON LGD.GEOG_DPTO_ID = LGL.GEOG_DPTO_ID
            INNER JOIN MSTRDB.DWH.LU_ARTC_ARTICULO AS LAA ON LAA.ARTC_ARTC_ID = FV.ARTC_ARTC_ID
        WHERE
            FV.TIEM_DIA_ID BETWEEN  DATEADD(MONTH, -6, CURRENT_DATE()) AND CURRENT_DATE()
            AND LGL.GEOG_UNNG_ID = 2
            AND LAA.ARTC_SECC_ID <> 42
            AND FFM.FDLN_MOVT_TIPO LIKE '%RP%'
        GROUP BY
            ALL
        )
    WHERE
        RNK = 1
        AND DOMINANTE IN (111, 152, 156, 158, 159, 160, 161, 162, 316, 125, 127, 317)
    ),

IMPACTADOS_FRONTERA AS
    (
    SELECT
        DISTINCT SOCI_SOCI_ID
    FROM
        SANDBOX_PLUS.DWH.SEGMENTOS_OFERTATA
    WHERE
        SOCI_SOCI_ID IN (SELECT SOCI_SOCI_ID FROM LOCAL_DOMINANTE)
    )

SELECT
    TO_VARCHAR(FFM.TIEM_DIA_ID, 'YYYY-MM') AS PERIODO,
    FFM.SOCI_SOCI_ID,
    FV.TICKET,
    FV.VNTA_IMPORTE_CON_IVA AS VENTA_CON_IVA,
    FV.VNTA_IMPORTE_SIN_IVA AS VENTA_SIN_IVA,
    FV.VNTA_UNIDADES * FV.VNTA_COSTO_PROM_POND AS COSTO
FROM
    MSTRDB.DWH.FT_VENTAS AS FV
    INNER JOIN MSTRDB.DWH.FT_FDLN_MOVIMIENTOS AS FFM ON FFM.TICKET = FV.TICKET
WHERE
    FFM.FDLN_MOVT_TIPO ILIKE '%RP%'
    AND FFM.GEOG_LOCL_ID IN (SELECT GEOG_LOCL_ID FROM MSTRDB.DWH.LU_GEOG_LOCAL WHERE GEOG_UNNG_ID = 2)
    AND YEAR(FFM.TIEM_DIA_ID) = 2023
    AND FFM.SOCI_SOCI_ID IN (SELECT * FROM IMPACTADOS_FRONTERA)
'''

cursor.execute(query)
comp = cursor.fetch_pandas_all()

comp['SOCI_SOCI_ID'] = comp['SOCI_SOCI_ID'].astype(str)
print(comp.shape[0])
comp.head(2)

19013822


,PERIODO,SOCI_SOCI_ID,TICKET,VENTA_CON_IVA,VENTA_SIN_IVA,COSTO
0,2023-01,327052,202301142483377784,30.44,27.67,20.382826
1,2023-01,112413,202301142482389257,49.85,45.32,25.564332


In [103]:
# Consumo
consumo = comp.groupby(['SOCI_SOCI_ID', 'PERIODO'])['VENTA_CON_IVA'].sum().reset_index()
consumo = consumo.groupby(['PERIODO'])['VENTA_CON_IVA'].median().reset_index()
consumo['VENTA_CON_IVA'].median()

3050.9975

In [106]:
# Frecuencia
frecuencia = comp.groupby(['SOCI_SOCI_ID', 'PERIODO'])['TICKET'].nunique().reset_index()
frecuencia = frecuencia.groupby(['PERIODO'])['TICKET'].median().reset_index()
frecuencia['TICKET'].median()

4.0

In [ ]:
# GB1 Porcentual
frecuencia = comp.groupby(['SOCI_SOCI_ID', 'PERIODO'])['TICKET'].nunique().reset_index()
frecuencia = frecuencia.groupby(['PERIODO'])['TICKET'].median().reset_index()
frecuencia['TICKET'].median()

# Impacto Ofertata

In [108]:
int(((comp['VENTA_SIN_IVA'].sum() - comp['COSTO'].sum()) / comp['VENTA_SIN_IVA'].sum()) * 100)

21

In [67]:
query = '''
WITH LOCAL_DOMINANTE AS
    (
    SELECT
        DISTINCT
        SOCI_SOCI_ID,
        DOMINANTE,
        GEOG_LOCL_DESC
    FROM
        (
        SELECT
            FFM.SOCI_SOCI_ID,
            LGL.GEOG_LOCL_COD AS DOMINANTE,
            LGL.GEOG_LOCL_DESC,
            SUM(FV.VNTA_IMPORTE_SIN_IVA) AS VENTAS,
            ROW_NUMBER() OVER (PARTITION BY SOCI_SOCI_ID ORDER BY VENTAS DESC) AS RNK
        FROM
            MSTRDB.DWH.FT_VENTAS AS FV
            INNER JOIN MSTRDB.DWH.FT_FDLN_MOVIMIENTOS AS FFM ON FV.TICKET = FFM.TICKET
            INNER JOIN MSTRDB.DWH.LU_GEOG_LOCAL AS LGL ON FV.GEOG_LOCL_ID = LGL.GEOG_LOCL_ID
            INNER JOIN MSTRDB.DWH.LU_GEOG_DEPTO AS LGD ON LGD.GEOG_DPTO_ID = LGL.GEOG_DPTO_ID
            INNER JOIN MSTRDB.DWH.LU_ARTC_ARTICULO AS LAA ON LAA.ARTC_ARTC_ID = FV.ARTC_ARTC_ID
        WHERE
            FV.TIEM_DIA_ID BETWEEN  DATEADD(MONTH, -6, CURRENT_DATE()) AND CURRENT_DATE()
            AND LGL.GEOG_UNNG_ID = 2
            AND LAA.ARTC_SECC_ID <> 42
            AND FFM.FDLN_MOVT_TIPO LIKE '%RP%'
        GROUP BY
            ALL
        )
    WHERE
        RNK = 1
        AND DOMINANTE IN (111, 152, 156, 158, 159, 160, 161, 162, 316, 125, 127, 317)
    ),

IMPACTADOS_FRONTERA AS
    (
    SELECT
        DISTINCT SOCI_SOCI_ID
    FROM
        SANDBOX_PLUS.DWH.SEGMENTOS_OFERTATA
    WHERE
        SOCI_SOCI_ID IN (SELECT SOCI_SOCI_ID FROM LOCAL_DOMINANTE)
    )

SELECT
    DISTINCT
    FFM.TIEM_DIA_ID,
    FFM.SOCI_SOCI_ID,
    IM.SUBCLASE
FROM
    MSTRDB.DWH.FT_VENTAS AS FV
    INNER JOIN MSTRDB.DWH.FT_FDLN_MOVIMIENTOS AS FFM ON FFM.TICKET = FV.TICKET
    INNER JOIN MSTRDB.DWH.LU_ARTC_ARTICULO AS LAA ON FV.ARTC_ARTC_ID = LAA.ARTC_ARTC_ID
    INNER JOIN MSTRDB.DWH.ITEM_MASTER AS IM ON IM.ITEM = LAA.ORIN
WHERE
    FFM.FDLN_MOVT_TIPO ILIKE '%RP%'
    AND FFM.GEOG_LOCL_ID IN (SELECT GEOG_LOCL_ID FROM MSTRDB.DWH.LU_GEOG_LOCAL WHERE GEOG_UNNG_ID = 2)
    AND FFM.TIEM_DIA_ID >= '2024-01-04'
    AND FFM.SOCI_SOCI_ID IN (SELECT * FROM IMPACTADOS_FRONTERA)
'''

cursor.execute(query)
trans = cursor.fetch_pandas_all()

trans['SOCI_SOCI_ID'] = trans['SOCI_SOCI_ID'].astype(str)
print(trans.shape[0])
trans.head(2)

1827055


,TIEM_DIA_ID,SOCI_SOCI_ID,SUBCLASE
0,2024-01-04,166295,2500050001
1,2024-01-04,84825,6300030002


In [68]:
trans_2 = trans[trans['SUBCLASE'].isin(clientes['SUBCLASE'].unique())]

## Clientes Activos - compraron la subclase?

In [69]:
activos = clientes[clientes['PERFIL_CLIENTE'] == 'ACTIVO'].merge(trans_2, on = ['SUBCLASE', 'SOCI_SOCI_ID'], how = 'inner')
activos.drop(['CLIE_TIPO_DOC', 'CLIE_CLIE_NDOC', 'CORREO', 'CELULAR'], axis = 1, inplace = True)
activos = activos[activos['FECHA'] < activos['TIEM_DIA_ID']]
activos[activos['SOCI_SOCI_ID'] == '241313']

,OFERTATA,FECHA,SUBCLASE,PERFIL_CLIENTE,SOCI_SOCI_ID,TIEM_DIA_ID
0,Ofertata 1,2024-01-04,1900010003,ACTIVO,241313,2024-02-17
18026,Ofertata 1,2024-01-04,8800030003,ACTIVO,241313,2024-01-07


## Comentarios

In [70]:
a = clientes[clientes['PERFIL_CLIENTE'] == 'ACTIVO'].shape[0]
b = clientes[clientes['PERFIL_CLIENTE'] == 'ACTIVO']['SOCI_SOCI_ID'].nunique()
c = activos.shape[0]
d = activos['SOCI_SOCI_ID'].nunique()
e = int(round(((c / a) * 100), 0))
f = int(round(((d / b) * 100), 0))

print('Tenemos dos criterios de segmentacion:')
print('')
print('A. Segmento Activos')
print('Para cada categoria (subclase) son los Clientes activos que no compran la subclase y tienen score 5')
print('')
print(f'Con los Ofertata 1, 4 y 7 hemos impactado a unos {b} Clientes con las subclases destacadas, siendo el total de impactos de {a}.')
print(f'Luego de ser impactados, unos {d} Clientes han realizado {c} compras de las subclases que se les promocionó.')
print('')
print(f'Compras / Impactos --> {e}%')
print(f'Clientes que compraron / Impactados --> {f}%')

Tenemos dos criterios de segmentacion:

A. Segmento Activos
Para cada categoria (subclase) son los Clientes activos que no compran la subclase y tienen score 5

Con los Ofertata 1, 4 y 7 hemos impactado a unos 38055 Clientes con las subclases destacadas, siendo el total de impactos de 273216.
Luego de ser impactados, unos 13663 Clientes han realizado 33638 compras de las subclases que se les promocionó.

Compras / Impactos --> 12%
Clientes que compraron / Impactados --> 36%


## Clientes Abandonadores

### Volvieron a TATA?

In [80]:
abandonador_2 = clientes[clientes['PERFIL_CLIENTE'] == 'ABANDONADOR']

g = abandonador_2['SOCI_SOCI_ID'].nunique()
h = trans[trans['SOCI_SOCI_ID'].isin(abandonador_2['SOCI_SOCI_ID'])]['SOCI_SOCI_ID'].nunique()
i = int(round(((h / g) * 100), 0))

### Volvieron y consumieron la subclase que se les promociono

In [89]:
abandonador_1 = clientes[clientes['PERFIL_CLIENTE'] == 'ABANDONADOR'].merge(trans_2, on = ['SUBCLASE', 'SOCI_SOCI_ID'], how = 'inner')
abandonador_1.drop(['CLIE_TIPO_DOC', 'CLIE_CLIE_NDOC', 'CORREO', 'CELULAR'], axis = 1, inplace = True)
abandonador_1 = abandonador_1[abandonador_1['FECHA'] < abandonador_1['TIEM_DIA_ID']]
abandonador_1
abandonador_1[abandonador_1['SOCI_SOCI_ID'] == '54490']

,OFERTATA,FECHA,SUBCLASE,PERFIL_CLIENTE,SOCI_SOCI_ID,TIEM_DIA_ID
8,Ofertata 1,2024-01-04,8800030003,ABANDONADOR,54490,2024-01-30
9,Ofertata 1,2024-01-04,7600060004,ABANDONADOR,54490,2024-01-30


In [96]:
j = abandonador_1['SOCI_SOCI_ID'].nunique()
k = int(round(((j / g) * 100), 0))

In [97]:
print('B. Segmento Abandonador')
print('Clientes que en el último mes no vengan a TATA y tengan Score 5 en la subclase')
print('')
print('Parte B.1. Clientes abandonadores que volvieron a TATA (en general)')
print(f'Fueron impactados {g} Clientes abandonadores, de los cuales volvieron a TATA unos {h}, es decir un {i}%')
print('')
print('Parte B.2. Clientes abandonadores que volvieron a TATA y consumieron las subclases promocionadas')
print(f'Fueron impactados {g} Clientes abandonadores, de los cuales volvieron a TATA (y consumieron la subclase propocionada) unos {j}, es decir un {k}%')

B. Segmento Abandonador
Clientes que en el último mes no vengan a TATA y tengan Score 5 en la subclase

Parte B.1. Clientes abandonadores que volvieron a TATA (en general)
Fueron impactados 22522 Clientes abandonadores, de los cuales volvieron a TATA unos 10806, es decir un 48%

Parte B.2. Clientes abandonadores que volvieron a TATA y consumieron las subclases promocionadas
Fueron impactados 22522 Clientes abandonadores, de los cuales volvieron a TATA (y consumieron la subclase propocionada) unos 1141, es decir un 5%


# Fin